
# 🧠 Fase 2 — Análise de Sentimentos (Português)  
**Projeto:** Mineração de Textos — Análise de Sentimentos  
**CRISP-DM:** Modeling / Evaluation  

Este notebook lê o CSV gerado na Fase 1, realiza **limpeza**, **detecção de idioma** e aplica um **modelo BERT-PT** para classificar **positivo/negativo/neutro**.  
Foi preparado para uso **autônomo** e para futura **integração com Flask** (sem `input()`), salvando a saída com **timestamp**.


In [ ]:

# ============================================================
# 1) Importações e parâmetros
# ============================================================
import os, re
import pandas as pd
import matplotlib.pyplot as plt
import emoji
from langdetect import detect, DetectorFactory
from transformers import pipeline
from glob import glob

DetectorFactory.seed = 0

INPUT_PATH = os.getenv("INPUT_PATH", "data/comentarios_coletados_*.csv")  # curinga tratado abaixo
OUTPUT_DIR = os.getenv("OUTPUT_DIR", "resultados")
HF_MODEL_PT = os.getenv("HF_MODEL_PT", "pierreguillou/bert-base-cased-sentiment-br")

print("✅ Bibliotecas carregadas.")

In [ ]:

# ============================================================
# 2) Descoberta automática do último arquivo de coleta
# ============================================================
def ultimo_csv_coleta(pattern: str = "data/comentarios_coletados_*.csv") -> str:
    arquivos = sorted(glob(pattern))
    if not arquivos:
        raise FileNotFoundError("Nenhum arquivo de coleta encontrado. Execute a Fase 1 antes.")
    return arquivos[-1]

if "*" in INPUT_PATH:
    INPUT_PATH = ultimo_csv_coleta(INPUT_PATH)

print(f"📥 Lendo dados de: {INPUT_PATH}")
df = pd.read_csv(INPUT_PATH)
print(f"Registros carregados: {len(df)}")
display(df.head(5))

In [ ]:

# ============================================================
# 3) Limpeza e normalização de texto
# ============================================================
def limpar_texto(texto: str) -> str:
    import re, emoji
    t = emoji.demojize(str(texto))
    t = re.sub(r"http\S+|www\.\S+", " ", t)
    t = re.sub(r"@[A-Za-z0-9_]+|#[A-Za-z0-9_]+", " ", t)
    t = re.sub(r"[^A-Za-zÀ-ÿ\s:]", " ", t)
    t = re.sub(r"\s+", " ", t).strip().lower()
    return t

df["comentario_limpo"] = df["comentario"].astype(str).apply(limpar_texto)
df = df[df["comentario_limpo"].str.len() > 3].copy()

print("🧹 Limpeza concluída.")
display(df[["video_titulo","comentario","comentario_limpo"]].head(5))

In [ ]:

# ============================================================
# 4) Detecção de idioma e filtro (PT)
# ============================================================
def detectar_idioma_seguro(texto: str) -> str:
    try:
        return detect(texto)
    except Exception:
        return "indefinido"

df["idioma"] = df["comentario_limpo"].apply(detectar_idioma_seguro)
df_pt = df[df["idioma"] == "pt"].copy()

print(f"Total após limpeza: {len(df)} | Em português: {len(df_pt)}")
display(df_pt.head(5))

In [ ]:

# ============================================================
# 5) Pipeline HuggingFace (PT-BR) — com batch
# ============================================================
sentiment_pt = pipeline(
    task="sentiment-analysis",
    model=HF_MODEL_PT,
    truncation=True,
    batch_size=32
)

def padronizar_label(lbl: str) -> str:
    m = {"POS": "positivo", "positive": "positivo",
         "NEG": "negativo", "negative": "negativo",
         "NEU": "neutro",   "neutral":  "neutro"}
    return m.get(lbl, lbl)

preds = sentiment_pt(df_pt["comentario_limpo"].tolist())
df_pt["sentimento"] = [padronizar_label(p["label"]) for p in preds]
df_pt["score"] = [p["score"] for p in preds]

print("✅ Análise concluída.")
display(df_pt[["video_titulo","comentario_limpo","sentimento","score"]].head(10))

In [ ]:

# ============================================================
# 6) Gráficos básicos
# ============================================================
if not df_pt.empty:
    dist = df_pt["sentimento"].value_counts().sort_index()
    plt.figure(figsize=(6,4))
    dist.plot(kind="bar")
    plt.title("Distribuição de Sentimentos (PT)")
    plt.xlabel("Sentimento")
    plt.ylabel("Quantidade")
    plt.tight_layout()
    plt.show()

    por_video = df_pt.groupby(["video_titulo","sentimento"]).size().unstack(fill_value=0)
    plt.figure(figsize=(10,5))
    por_video.plot(kind="bar", stacked=True)
    plt.title("Sentimentos por Vídeo (PT)")
    plt.xlabel("Vídeo")
    plt.ylabel("Quantidade")
    plt.tight_layout()
    plt.show()
else:
    print("Sem dados em PT para gráficos.")

In [ ]:

# ============================================================
# 7) Exportação com timestamp
# ============================================================
os.makedirs(OUTPUT_DIR, exist_ok=True)
ts = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
out_path = os.path.join(OUTPUT_DIR, f"comentarios_analisados_{ts}.csv")
df_pt.to_csv(out_path, index=False, encoding="utf-8-sig")

print(f"📦 Salvo em: {out_path} | Linhas: {len(df_pt)}")